In [2]:
import os
import cv2
import ray
import warnings
import numpy as np 
import pandas as pd
import mediapipe as mp

from tqdm import tqdm
warnings.filterwarnings('ignore')

In [3]:
base_dir = "C:/Users/NEULET/Desktop/dataset"

video_src = os.path.join(base_dir, "source")
keypoints_src = os.path.join(base_dir, "keypoints")

In [4]:
def get_max_length(keypoints_list):
    max_length = 0  # val of max keypoints length / 키포인트 최대 길이값
    for keypoints_data in keypoints_list:
        current_length = keypoints_data.shape[0]    # keypoints length / 현재 keypoints의 길이값
        if current_length > max_length:
            max_length = current_length
    print(f"Max length : {max_length}")    
    return int(max_length)

In [6]:
@ray.remote
def extract_keypoints(video_path):
    # mediapipe pose model init / mediapipe pose model 선언
    mp_pose = mp.solutions.pose
    pose_model = mp_pose.Pose(static_image_mode=True)
    
    filename = os.path.splitext(os.path.basename(video_path))[0]  # Extract filename
    cap = cv2.VideoCapture(video_path)
    pose_keypoints = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = pose_model.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            keypoints = [[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]
            pose_keypoints.append(keypoints)

    cap.release()
    keypoints_array = np.array(pose_keypoints)
    return filename, keypoints_array
    
def ray_extract_keypoints(video_src):
    ray.init(num_cpus=6)  # initalize ray / ray 인스턴스 선언

    video_paths = [os.path.join(video_src, video_file) for video_file in os.listdir(video_src)]
    futures = [extract_keypoints.remote(video_path) for video_path in video_paths]
    
    results = [ray.get(f) for f in tqdm(futures)]
    filename_list, pose_keypoints_list = zip(*results)

    match_df = pd.DataFrame({
        'filename': filename_list,
        'pose_keypoints': pose_keypoints_list
    })
    
    ray.shutdown()
    return match_df

In [ ]:
match_df = ray_extract_keypoints(video_src)
max_length = get_max_length(match_df['pose_keypoints'])

for i, keypoints_data in enumerate(tqdm(match_df['pose_keypoints'])):
    filename = match_df['filename'][i]  # corrected indexing / 인덱싱 수정
    current_length = keypoints_data.shape[0]

    if max_length > current_length:
        padding = np.zeros((max_length - current_length, 33, 3))
        padded_keypoints = np.vstack((keypoints_data, padding))
    else:
        padded_keypoints = keypoints_data

    np.save(keypoints_src + f"/{filename}_padded.npy", padded_keypoints)
    print(f"{filename} - {padded_keypoints.shape}")

(extract_keypoints pid=17672) W0000 00:00:1731372306.859650   45184 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors. [repeated 10x across cluster]
